## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Strezhevoy,RU,60.7333,77.5889,23.77,97,85,4.74,light snow
1,1,New Norfolk,AU,-42.7826,147.0587,47.71,83,18,2.48,few clouds
2,2,Rikitea,PF,-23.1203,-134.9692,79.41,71,7,15.84,clear sky
3,3,Albany,US,42.6001,-73.9662,47.64,91,100,10.00,moderate rain
4,4,Broome,US,42.2506,-75.8330,48.38,83,100,1.01,moderate rain
5,5,Carhuaz,PE,-9.2819,-77.6447,64.53,73,98,2.44,light rain
6,6,Portoferraio,IT,42.8115,10.3146,56.52,79,8,14.12,clear sky
7,7,Florianopolis,BR,-27.6146,-48.5012,72.70,78,75,6.91,broken clouds
8,8,Busselton,AU,-33.6500,115.3333,63.84,64,85,7.05,overcast clouds
9,9,East London,ZA,-33.0153,27.9116,61.48,64,18,16.28,few clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,79.41,71,7,15.84,clear sky
7,7,Florianopolis,BR,-27.6146,-48.5012,72.70,78,75,6.91,broken clouds
10,10,San Patricio,US,28.0170,-97.5169,82.17,14,0,15.48,clear sky
12,12,Cidreira,BR,-30.1811,-50.2056,69.96,89,100,9.33,overcast clouds
13,13,Mahebourg,MU,-20.4081,57.7000,76.24,83,46,14.18,light rain
17,17,Byron Bay,AU,-28.6500,153.6167,66.70,85,95,10.33,overcast clouds
18,18,Samana,DO,19.2056,-69.3369,84.42,79,20,6.91,few clouds
19,19,Ambilobe,MG,-13.2000,49.0500,74.71,94,98,1.74,overcast clouds
20,20,Porbandar,IN,21.6422,69.6093,83.34,55,34,9.80,scattered clouds
29,29,Avarua,CK,-21.2078,-159.7750,86.05,74,40,12.66,scattered clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.41,clear sky,-23.1203,-134.9692,
7,Florianopolis,BR,72.70,broken clouds,-27.6146,-48.5012,
10,San Patricio,US,82.17,clear sky,28.0170,-97.5169,
12,Cidreira,BR,69.96,overcast clouds,-30.1811,-50.2056,
13,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,
17,Byron Bay,AU,66.70,overcast clouds,-28.6500,153.6167,
18,Samana,DO,84.42,few clouds,19.2056,-69.3369,
19,Ambilobe,MG,74.71,overcast clouds,-13.2000,49.0500,
20,Porbandar,IN,83.34,scattered clouds,21.6422,69.6093,
29,Avarua,CK,86.05,scattered clouds,-21.2078,-159.7750,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,79.41,clear sky,-23.1203,-134.9692,People ThankYou
7,Florianopolis,BR,72.70,broken clouds,-27.6146,-48.5012,Hotel Boutique Quinta das Videiras
10,San Patricio,US,82.17,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
12,Cidreira,BR,69.96,overcast clouds,-30.1811,-50.2056,Hotel Castelo
13,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
17,Byron Bay,AU,66.70,overcast clouds,-28.6500,153.6167,Backpackers Inn
18,Samana,DO,84.42,few clouds,19.2056,-69.3369,The Bannister - Hotel & Yacht Club
19,Ambilobe,MG,74.71,overcast clouds,-13.2000,49.0500,Hôtel & Restaurant DIANA Ambilobe
20,Porbandar,IN,83.34,scattered clouds,21.6422,69.6093,Lords Inn Porbandar
29,Avarua,CK,86.05,scattered clouds,-21.2078,-159.7750,Paradise Inn


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [38]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))